In [29]:
import pandas as pd
import numpy as np
import warnings
from sklearn.metrics.pairwise import cosine_similarity
from sklearn.preprocessing import MinMaxScaler
# Disable all warnings
warnings.filterwarnings("ignore")

In [30]:
file_path = r"C:\Users\Hp\Desktop\ARSFULL.xlsx"
excel_file = pd.ExcelFile(file_path)

# Get the sheet names
sheet_names = excel_file.sheet_names

# Load each sheet into a separate DataFrame
df1 = pd.read_excel(file_path, sheet_name=sheet_names[0])
df2 = pd.read_excel(file_path, sheet_name=sheet_names[1])
df3 = pd.read_excel(file_path, sheet_name=sheet_names[2])
df4 = pd.read_excel(file_path, sheet_name=sheet_names[3])
df5 = pd.read_excel(file_path, sheet_name=sheet_names[4])
df6 = pd.read_excel(file_path, sheet_name=sheet_names[5])


dfs = [df1 , df2 , df3 , df4 , df5 , df6]

In [31]:
for i in dfs:
    
    print('\n' ,i['Recommended Areas'].value_counts())



 All Irrigated Areas of Punjab                61
All Irrigated and Barani  Areas of Punjab     1
Name: Recommended Areas, dtype: int64

 All Areas of Punjab                           66
DG Khan & Bahawalpur Divisions                 3
Faisalabad, DG Khan & Bahawalpur Divisions     1
Name: Recommended Areas, dtype: int64

 Whole Punjab                                                                     19
Other than riverine areas of Punjab                                               4
                                                                                  2
Whole Punjab including river belt areas                                           2
Southern Punjab only other than riverine areas                                    1
Whole Punjab other than river belt areas and particularly for southern Punjab     1
Whole Punjab other than river belt areas                                          1
Name: Recommended Areas, dtype: int64

 All Punjab     30
All Punjab      1
Name: Recom

In [32]:
def pad_na_with_next(df, column_name):
    df[column_name] = df[column_name].fillna(method='bfill')
    return df


def clean_and_split_areas(area_string):
    if isinstance(area_string, str):  # Check if the value is a string
        # Remove '&' and split by comma
        cleaned_areas = area_string.replace('&', ',').split(',')
        # Strip any leading/trailing whitespace from each area
        cleaned_areas = [area.strip() for area in cleaned_areas]
        return cleaned_areas
    else:
        return []  # Return an empty list for non-string values


df5['Recommended Areas'] = df5['Recommended Areas'].apply(clean_and_split_areas)

In [33]:
df6 = df5.drop('Salient Features' , axis = 1)
df6 = pad_na_with_next(df6, 'Average Yield (ton/ha)')
df6 = df6.dropna()


df5 = df5.dropna()

df4 = df4.dropna()


df3 = df3.drop('Unnamed: 9' , axis = 1)
df3 = df3.drop("Salient Features" , axis = 1)
df3 = df3.dropna()


In [34]:
def parse_features(df):
    # Initialize new columns with default values
    df['yield'] = 'high'
    df['skin color'] = None
    df['maturing'] = None
    df['suitable for processing industry'] = 'no'
    df['climate resilient'] = 'no'
    
    # Loop through each row and update the new columns based on the features
    for index, row in df.iterrows():
        df['Salient Features'] = df['Salient Features'].str.replace(' and ', ', ').str.replace('&', ',')
        features = row['Salient Features'].lower()

        if 'high yielding' in features:
            df.at[index, 'yield'] = 'high'
        elif 'medium yielding' in features:
            df.at[index, 'yield'] = 'medium'
        elif 'low yielding' in features:
            df.at[index, 'yield'] = 'low'
        
        if 'red skinned' in features:
            df.at[index, 'skin color'] = 'red'
        elif 'yellow skinned' in features:
            df.at[index, 'skin color'] = 'yellow'
        elif 'white skinned' in features:
            df.at[index, 'skin color'] = 'white'
        
        if 'late maturing' in features:
            df.at[index, 'maturing'] = 'late'
        else:
            df.at[index, 'maturing'] = 'early'

        if 'suitable for processing industry' in features:
            df.at[index, 'suitable for processing industry'] = 'yes'
        
        if 'climate resilient' in features:
            df.at[index, 'climate resilient'] = 'yes'
    
    return df

df5 = parse_features(df5)


skin_color_mapping = {'red': 0, 'white': 1, 'yellow': 2}
maturity_mapping = {'late': 0, 'early': 1}
processing_industry_mapping = {'yes': 1, 'no': 0}
climate_resilient_mapping = {'yes': 1, 'no': 0}

# Apply mapping using map function
df5['skin color'] = df5['skin color'].map(skin_color_mapping)
df5['maturing'] = df5['maturing'].map(maturity_mapping)
df5['suitable for processing industry'] = df5['suitable for processing industry'].map(processing_industry_mapping)
df5['climate resilient'] = df5['climate resilient'].map(climate_resilient_mapping)

df5

,Sr. No.,Variety Name,Parentage,Year of Release,Sowing Time,Average Yield (ton/ha),Recommended Areas,Salient Features,yield,skin color,maturing,suitable for processing industry,climate resilient
2,2.0,Sahiwal White,FD 35-36 x SH-5,2019.0,2nd fortnight of October,30.5,"[Sahiwal, Okara, Pakpattan, Kasur]","High yielding , white skinned, climate resilie...",high,1,1,0,1
4,3.0,Cosmo,FD 3-15 x FD 35-36,2019.0,2nd fortnight of October,29.9,"[Sahiwal, Okara, Pakpattan, Kasur]","High yielding , yellow skinned , tolerant to...",high,2,1,0,0
6,4.0,Sialkot Red,SH-5 x Cardinal,2018.0,2nd fortnight of October,25.8,"[Sialkot, Gujranwala, Sahiwal, Okara, Pakpatta...","High yielding , red skinned, climate resilient...",high,0,1,0,1
8,5.0,Sadaf,FD 3-15 x FD 35-36,2016.0,2nd fortnight of October,26.6,"[Sahiwal, Okara, Pakpattan, Kasur]","High yielding , white skinned, climate resilie...",high,1,1,1,1
10,6.0,Ruby,384636-1 x FD 1-8,2016.0,2nd fortnight of October,27.3,"[Sahiwal, Okara, Pakpattan, Kasur]","High yielding , red skinned, late maturing, t...",high,0,0,1,0
12,7.0,PRI-Red,FD 44-24 x FD 12-24,2013.0,2nd fortnight of October,34.2,"[Sahiwal, Okara, Pakpattan, Kasur]","High yielding , red skinned, late maturing, to...",high,0,0,0,0
14,8.0,SH-5,Cardinal x Desiree,2006.0,2nd fortnight of October,35.0,"[Sialkot, Gujranwala, Sahiwal, Okara, Pakpatta...","High yielding , red skinned, late maturing, to...",high,0,0,0,0


In [35]:
# Define the lists with all city names in lowercase

# List of irrigated areas of Punjab
irrigated_areas = [
    "lahore", "faisalabad", "multan", "bahawalpur", "sargodha",
    "sialkot", "sheikhupura", "rahim yar khan", "jhang", "gujrat",
    "sahiwal", "wah cantonment", "kasur", "okara", "chiniot",
    "kamoke", "hafizabad", "sadiqabad", "burewala", "khanewal",
    "mandi bahauddin", "khanpur", "pakpattan", "daska", "gojra",
    "muridke", "bahawalnagar", "samundri", "jaranwala", "chishtian",
    "vehari", "kot abdul malik", "ferozewala", "gujranwala cantonment",
    "kamalia", "ahmedpur east", "wazirabad", "lodhran", "hasilpur",
    "arifwala", "sambrial", "haroonabad", "narowal", "bhalwal"
]

# List of barani areas of Punjab
barani_areas = [
    "rawalpindi", "attock", "chakwal", "jhelum", "khushab",
    "mianwali", "taxila"
]

# List of all areas of Punjab
all_punjab = irrigated_areas + barani_areas + [
    "dera ghazi khan", "layyah", "muzaffargarh", "kot addu"
]

whole_punjab = irrigated_areas + barani_areas + [
    "dera ghazi khan", "layyah", "muzaffargarh", "kot addu"
]

# DG Khan & Bahawalpur Divisions
dg_khan_bahawalpur = [
    "dera ghazi khan", "layyah", "muzaffargarh", "kot addu",
    "bahawalpur", "rahim yar khan", "sadiqabad", "ahmedpur east",
    "hasilpur", "haroonabad", "bahawalnagar"
]

# Faisalabad, DG Khan & Bahawalpur Divisions
faisalabad_dgkhan_bahawalpur = dg_khan_bahawalpur + ["faisalabad"]

# Other categories
non_riverine_punjab = [
    "rawalpindi", "sargodha", "attock", "chakwal", "jhelum", "khushab"
] + [city for city in dg_khan_bahawalpur if city not in ["rahim yar khan", "muzaffargarh", "kot addu", "sadiqabad"]]

sahiwal_okara_pakpattan_kasur = ["sahiwal", "okara", "pakpattan", "kasur"]
sialkot_gujranwala_sahiwal_okara_pakpattan_kasur = ["sialkot", "gujranwala"] + sahiwal_okara_pakpattan_kasur

# Classification function
def classify_city(city):
    city = city.lower()
    classification = []
    
    if city in whole_punjab:
        classification.append("Whole Punjab")
    
    if city in irrigated_areas:
        classification.append("Irrigated Areas of Punjab")
    
    if city in barani_areas:
        classification.append("Barani Areas of Punjab")
    
    if city in all_punjab:
        classification.append("All Punjab")
    
    if city in dg_khan_bahawalpur:
        classification.append("DG Khan & Bahawalpur Divisions")
    
    if city in faisalabad_dgkhan_bahawalpur:
        classification.append("Faisalabad, DG Khan & Bahawalpur Divisions")
    
    if city in non_riverine_punjab:
        classification.append("Other than riverine areas of Punjab")
    
    if city in sahiwal_okara_pakpattan_kasur:
        L = ['Sahiwal', 'Okara' , 'Pakpattan' , 'Kasur']
        for i in L:
            classification.append(i)
    
    if city in sialkot_gujranwala_sahiwal_okara_pakpattan_kasur:
        L = ['Sialkot', 'Gujranwala', 'Sahiwal', 'Okara', 'Pakpattan', 'Kasur']
        for i in L:
            classification.append(i)
    
    return classification



In [36]:
df1.rename(columns={'Yield Potential (Maund/Acre)': 'yield'}, inplace=True)
df2.rename(columns={'Yield Potential (Maunds/Acre)': 'yield'}, inplace=True)
df3.rename(columns={'Av. yield': 'yield'}, inplace=True)
df4.rename(columns={'Potential Yield (Maund/Acre)': 'yield'}, inplace=True)
df5.rename(columns={'Average Yield (ton/ha)': 'yield'}, inplace=True)
df6.rename(columns={'Average Yield (ton/ha)': 'yield'}, inplace=True)

In [37]:
# Rename the variety columns to 'variety'
df1.rename(columns={'Varieties': 'variety'}, inplace=True)
df2.rename(columns={'Varieties': 'variety'}, inplace=True)
df3.rename(columns={'Variety Name': 'variety'}, inplace=True)
df4.rename(columns={'Variety Name': 'variety'}, inplace=True)
df5.rename(columns={'Variety Name': 'variety'}, inplace=True)
df6.rename(columns={'Variety Name': 'variety'}, inplace=True)

In [38]:
# Convert the 'yield' columns to numeric, coercing errors to NaN
df1['yield'] = pd.to_numeric(df1['yield'], errors='coerce')
df2['yield'] = df2['yield'].apply(lambda x: x.split()[0] if isinstance(x, str) else x)
df2['yield'] = pd.to_numeric(df2['yield'], errors='coerce')
df3['yield'] = pd.to_numeric(df3['yield'], errors='coerce')
df4['yield'] = pd.to_numeric(df4['yield'], errors='coerce')
df6['yield'] = pd.to_numeric(df6['yield'], errors='coerce')

In [39]:
def determine_growable_crops():
    crops = {}
    ddf5 = ['Sahiwal', 'Okara' ,'Pakpattan & Kasur']
    
    if not df1[df1['Recommended Areas'].apply(lambda x: any(city in str(x) for city in classification))].empty:
        crops['wheat'] = df1
    if not df2[df2['Recommended Areas'].apply(lambda x: any(city in str(x) for city in classification))].empty:
        crops['cotton'] = df2
    if not df3[df3['Recommended Areas'].apply(lambda x: any(city in str(x) for city in classification))].empty:
        crops['sugarcane'] = df3
    if not df4[df4['Recommended Areas'].apply(lambda x: any(city in str(x) for city in classification))].empty:
        crops['rice'] = df4
    if not df5[df5['Recommended Areas'].apply(lambda x: any(city in str(x) for city in classification))].empty:
        crops['potato'] = df5
    if not df6[df6['Recommended Areas'].apply(lambda x: any(city in str(x) for city in classification))].empty:
        crops['groundnut'] = df6
    return crops

In [40]:
def filter_records(df, attribute_ranges):
    filtered_df = df.copy()
    for attr, (lower, upper) in attribute_ranges.items():
        print(attr)
        filtered_df = filtered_df[(filtered_df[attr] >= lower) & (filtered_df[attr] <= upper)]
    return filtered_df

In [41]:

def cosine_sim(filtered_df, user_ranges):
    # Convert the user-defined ranges dictionary to a DataFrame with one record
    l = pd.DataFrame(user_ranges)
    average_record = l.mean(axis=0)
    # Convert the average record to a DataFrame
    average_df = pd.DataFrame(average_record).T
    
    # Extract the specified features from the filtered DataFrame
    selected_features = list(user_ranges.keys())
    filtered_records = filtered_df[selected_features]
    
    # Calculate cosine similarity for each record
    similarity_scores = []
    for index, row in filtered_records.iterrows():
        similarity_score = cosine_similarity(average_df, filtered_records)
        similarity_scores.append(similarity_score[0][0])
    
    # Add similarity scores as a new column to the filtered DataFrame
    filtered_df['Cosine Similarity'] = similarity_scores
    
    # Get top 3 records with highest similarity scores
    top_3_records = filtered_df.nlargest(3, 'Cosine Similarity')
    
    return top_3_records


from sklearn.metrics.pairwise import cosine_similarity

def cosinedf5( attr):
    # Define mappings
    sims = []
    attr_ranges_df = pd.DataFrame(attr, index=[0])  
    
    attr_ranges_df['skin color'] = attr_ranges_df['skin color'].map(skin_color_mapping)
    attr_ranges_df['maturing'] = attr_ranges_df['maturing'].map(maturity_mapping)
    attr_ranges_df['suitable for processing industry'] = attr_ranges_df['suitable for processing industry'].map(processing_industry_mapping)
    attr_ranges_df['climate resilient'] = attr_ranges_df['climate resilient'].map(climate_resilient_mapping)


    encoded_df5 = df5[['skin color', 'maturing', 'suitable for processing industry', 'climate resilient']]
    similarities = cosine_similarity(attr_ranges_df, encoded_df5)
    sims.append(similarities)
    flat_sims = np.array(sims).flatten()  # Flatten the similarity scores
    top_indices = np.argsort(flat_sims)[::-1][:3]

    results_df = pd.DataFrame(columns=df5.columns)  # Initialize an empty DataFrame

    for idx in top_indices:
        row_data = df5.iloc[idx].tolist()  # Get the row data without similarity score
        results_df = results_df.append(pd.Series(row_data, index=results_df.columns), ignore_index=True)

    results_df['Cosine Similarity'] = flat_sims[top_indices]  # Add the similarity scores separately

    return results_df



In [42]:
def top_3_yield(df):
    # Ensure the 'Yield Potential (Maund/Acre)' column exists
    if 'Yield Potential (Maund/Acre)' not in df.columns:
        raise ValueError("The DataFrame does not have a 'Yield Potential (Maund/Acre)' column.")

    # Sort the DataFrame by 'Yield Potential (Maund/Acre)' in descending order and get the top 3 records
    top_3_records = df.sort_values(by='Yield Potential (Maund/Acre)', ascending=False).head(3)

    return top_3_records

In [43]:
def inputs(selected_crop):
    if selected_crop == 'wheat':
        attr_ranges = {}
        attr_ranges['1000 Kernel Weight (g)'] = tuple(map(float, input("Enter range for 1000 Kernel Weight (g), separated by space: ").split()))
        attr_ranges['Test Weight (kg/hl)'] = tuple(map(float, input("Enter range for Test Weight (kg/hl), separated by space: ").split()))
        attr_ranges['Protein %'] = tuple(map(float, input("Enter range for Protein %, separated by space: ").split()))
        attr_ranges['Bread Baking Quality Scores'] = tuple(map(float, input("Enter range for Bread Baking Quality Scores, separated by space: ").split()))
        #filtered_df1 = filter_records(df1, attr_ranges)
        records = cosine_sim(df1 , attr_ranges)
        

    elif selected_crop == 'cotton':
        attr_ranges = {}
        attr_ranges['G.O.T (%)'] = tuple(map(float, input("Enter range for G.O.T (%), separated by space: ").split()))
        attr_ranges['SL (mm)'] = tuple(map(float, input("Enter range for SL (mm), separated by space: ").split()))
        attr_ranges['Mike (μg/inch)'] = tuple(map(float, input("Enter range for  Mike (μg/inch), separated by space: ").split()))
        attr_ranges['Strength (tppsi)'] = tuple(map(float, input("Enter range for Strength (tppsi), separated by space: ").split()))
        # filtered_df2 = filter_records(df2, attr_ranges)
        records = cosine_sim(df2 , attr_ranges)


    elif selected_crop == 'sugarcane':
            attr_ranges = {
                'Sugar recovery (%)': tuple(map(float, input("Enter range for Sugar recovery (%), separated by space: ").split())),
                'Sugar yield': tuple(map(float, input("Enter range for Sugar yield, separated by space: ").split()))
            }
            # filtered_df3 = filter_records(df3, attr_ranges)
            records = cosine_sim(df3, attr_ranges)

        
    elif selected_crop == 'rice':
        attr_ranges = {}
        attr_ranges['Maturity (Days)'] = tuple(map(float, input("Enter range for Maturity (Days), separated by space: ").split()))
        attr_ranges['Plant Height (cm)'] = tuple(map(float, input("Enter range for Plant Height (cm), separated by space: ").split()))
        attr_ranges['Grain Length (mm)'] = tuple(map(float, input("Enter range for  Grain Length (mm), separated by space: ").split()))
        # filtered_df4 = filter_records(df4, attr_ranges)
        records = cosine_sim(df4 , attr_ranges)
        
    elif selected_crop == 'potato':
        attr_ranges = {
            'skin color':input("Enter skin color(red/yellow/white): ").lower(),
            'suitable for processing industry': input("Enter suitable for processing industry (Yes/No) ").lower() ,
            'maturing' : input('Enter maturing Time (late/early): ').lower(),
            'climate resilient': input("Enter climate resilient (Yes/No):").lower()
        }
        
        records = cosinedf5(attr_ranges)
        
    elif selected_crop == 'groundnut':
        records = top_3_yield(df6)
        
    print(attr_ranges)

        
    return records

In [45]:
# Example usage
city_name = input("Enter the city name: ")
classification = classify_city(city_name) # this to identify city class call this function after user input city

if classification:
    print(f"The city '{city_name}' belongs to the following classes: {', '.join(classification)}")
else:
    print(f"The city '{city_name}' does not belong to any of the specified classes.")


The city 'lahore' belongs to the following classes: Whole Punjab, Irrigated Areas of Punjab, All Punjab


In [46]:
growable_crops = determine_growable_crops()
for crop in growable_crops.keys():
    print(crop.capitalize())   

#these will be shown in 2nd drop down menu


Wheat
Sugarcane
Rice


In [47]:
import sys
# ye krnay ki zaroorat nhi cuz wo drop down se selet krega
# so condition ki need nhi
if growable_crops:
    selected_crop = input("Please enter the crop you want to grow: ").lower()
    if selected_crop in growable_crops:
        print("Answer stored.")

    else:
        print("Invalid crop selected. Please try again.")
else:
    print("No crops can be grown in the specified areas based on the given classification.")

Answer stored.


In [48]:
out = inputs(selected_crop) # call this for input u might have to change this for gui 
out # ye final output hoga


{'1000 Kernel Weight (g)': (1.0, 1.0), 'Test Weight (kg/hl)': (1.0, 1.0), 'Protein %': (1.0, 1.0), 'Bread Baking Quality Scores': (1.0, 1.0)}


,variety,Year of Release,Sowing Time,yield,Recommended Areas,1000 Kernel Weight (g),Test Weight (kg/hl),Protein %,Bread Baking Quality Scores,Chapati Making Quality,Cosine Similarity
0,Arooj 2022,2022,1st November-20th November,76,All Irrigated and Barani Areas of Punjab,45.0,72.0,12.5,70.0,Good,0.900689
1,Subhani 2021,2021,5th November-20th November,70,All Irrigated Areas of Punjab,47.0,74.0,14.0,71.0,Very Good,0.900689
2,MH 2021,2021,1st November-20th November,70,All Irrigated Areas of Punjab,41.0,78.0,13.0,71.0,Very Good,0.900689
